<a href="https://colab.research.google.com/github/Chaotic-Legend/CMP-414-Codes/blob/main/Homework%20%235%3A%20Multi-Linear%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework #5: Multi-Linear Regression

For this homework assignment, you will build a linear regression model with multiple input features. Below is a example dataset of house prices. The objective is to predict the price of a house based on its size (sq. ft.), number of bedrooms, and age (years).

In [1]:
import torch

# Features: size (sq. ft.), bedrooms, age (years)
X = torch.tensor([
    [1500, 3, 10], [1800, 4, 5], [1200, 2, 20],
    [2200, 4, 2], [1650, 3, 8], [3000, 5, 1],
    [1100, 2, 30], [2500, 4, 3], [1900, 3, 12]
], dtype=torch.float32)

# Target: price (in $1000s)
y = torch.tensor([300, 380, 250, 450, 350, 550, 220, 500, 400],
                dtype=torch.float32)

The size values are much greater than values from the other two features. To avoid having size dominant in the model, we need to put these values to the same scale. This process is called **data normalization**.

In [2]:
from sklearn.preprocessing import StandardScaler
# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = torch.tensor(X_scaled, dtype=torch.float32)

1. Initialize Parameters: Create a weight w1, w2, w3 and a bias b that require gradients. Initialize them as a torch tensor with random values (e.g., torch.randn). Remember to set `requires_grad=True`.

In [3]:
# Initialize weights (w0, w1, w2) and bias (b)
# Using random values that require gradient computation
w = torch.randn(3, requires_grad=True, dtype=torch.float32)
b = torch.randn(1, requires_grad=True, dtype=torch.float32)

print("=== Initial Parameters ===")
print("w =", w)
print("b =", b)

=== Initial Parameters ===
w = tensor([-0.3712,  0.0477, -0.1444], requires_grad=True)
b = tensor([0.4239], requires_grad=True)


2. Define the Model: The linear model calculates the prediction as follows:

prediction = w0 * size + w1 * bedrooms + w2 * age + b

Write a simple function model(X) that calculates the predicted y_pred using the expression:

`y_pred=X[:, 0] * w0 + X[:, 1] * w1 + X[:, 2] * w2 + b`

In [4]:
# Linear model: y_pred = w0*x1 + w1*x2 + w2*x3 + b
def model(X):
    return X @ w + b

# Test model output before training
print("=== Initial Predictions ===")
print(model(X_scaled.to(torch.float32)))

=== Initial Predictions ===
tensor([ 0.6475,  0.5861,  0.6286,  0.3778,  0.5834, -0.0686,  0.5320,  0.1693,
         0.3588], grad_fn=<AddBackward0>)


3. Define Loss Function and Optimizer:

- Use Mean Squared Error (MSE) as your loss function
- Use Stochastic Gradient Descent (SGD) as your optimizer. torch.optim.SGD will work.

In [5]:
# Mean Squared Error (MSE) Loss
loss_fn = torch.nn.MSELoss()

# Stochastic Gradient Descent (SGD) Optimizer
optimizer = torch.optim.SGD([w, b], lr=0.01)

4. Training Loop:

In a loop:
* a. Calculate the model's prediction (y_pred).
* b. Compute the loss.
* c. Zero out the gradients with optimizer.zero_grad()
* d. Calculate the gradients using loss.backward().
* e. Update the parameters using optimizer.step().

Execute the training loop with an proper number of epochs. Print the loss during training to observe the training progress.

In [6]:
print("=== Training Progress ===")
epochs = 500
for epoch in range(epochs):
    # Forward pass: compute predictions
    y_pred = model(X_scaled)

    # Compute loss
    loss = loss_fn(y_pred, y)

    # Zero gradients before backward pass
    optimizer.zero_grad()

    # Backward pass: compute gradients
    loss.backward()

    # Update parameters
    optimizer.step()

    # Print progress every 50 epochs with a title
    if (epoch + 1) % 50 == 0: print(f"Epoch {epoch+1}/{epochs}, Loss = {loss.item():.4f}\n")

=== Training Progress ===
Epoch 50/500, Loss = 20297.8125

Epoch 100/500, Loss = 3115.0229

Epoch 150/500, Loss = 786.6086

Epoch 200/500, Loss = 436.2777

Epoch 250/500, Loss = 357.6859

Epoch 300/500, Loss = 321.9916

Epoch 350/500, Loss = 297.0341

Epoch 400/500, Loss = 277.2621

Epoch 450/500, Loss = 261.0227

Epoch 500/500, Loss = 247.4362



5. Results: After the loop, print the final learned values for w and b.

In [7]:
print("=== Final Learned Parameters ===")
feature_names = ["Size", "Bedrooms", "Age"]
weights = w.detach().numpy()
bias_val = b.item()

# Print each weight with its corresponding feature
for name, weight in zip(feature_names, weights):
    print(f"w_{name}: {weight:.4f}")
print(f"Bias: {bias_val:.4f}")

# Predict house prices after training
predicted = model(X_scaled.to(torch.float32)).detach().numpy()

# Print predicted prices
print("\n=== Predicted House Prices (in $1000s) ===")
for i, price in enumerate(predicted, start=1): print(f"House {i}: {price:.1f}\n")

=== Final Learned Parameters ===
w_Size: 71.2383
w_Bedrooms: 17.9257
w_Age: -16.2900
Bias: 377.7623

=== Predicted House Prices (in $1000s) ===
House 1: 325.8

House 2: 390.8

House 3: 251.8

House 4: 445.5

House 5: 347.9

House 6: 564.8

House 7: 221.3

House 8: 480.6

House 9: 371.4

